In [9]:
from langchain_community.tools import YouTubeSearchTool
from crewai import Task, Agent, Process, Crew
from crewai_tools import SerperDevTool
import yaml
from dotenv import load_dotenv
load_dotenv()
import os
from crewai_tools import tool
import requests
YOUTUBE_API_KEY = os.getenv('YOUTUBE_API_KEY')

In [2]:
from crewai import LLM
llm = LLM(model="groq/llama-3.1-70b-versatile")

In [22]:
@tool
def youtube_channel_search_tool(query: str, max_results: int = 1) -> str:
    """
    Cerca video su YouTube solo da una lista specifica di canali.

    Args:
        query (str): Una versione semplificata e in inglese, adatta per una ricerca su youtube, della query di ricerca fornita dall'utente.
        max_results (int): Numero massimo di risultati da restituire.

    Returns:
        str: Il link al primo video trovato, o un messaggio che indica che non sono stati trovati video.
    """
    YOUTUBE_SEARCH_URL = "https://www.googleapis.com/youtube/v3/search"
    allowed_channels=['UCwywRelPfy7U8jAI312J_Xw']
    # Effettua la ricerca per ogni canale consentito
    for channel_id in allowed_channels:
        params = {
            "part": "snippet",
            "q": query,
            "channelId": channel_id,
            "maxResults": max_results,
            "type": "video",
            "key": YOUTUBE_API_KEY,
        }

        response = requests.get(YOUTUBE_SEARCH_URL, params=params)
        data = response.json()

        # Controlla se ci sono risultati
        if "items" in data and len(data["items"]) > 0:
            # Restituisci il link al primo video trovato
            video_id = data["items"][0]["id"]["videoId"]
            return f"https://www.youtube.com/watch?v={video_id}"

    # Se nessun video è trovato
    return "Nessun video pertinente trovato per la query specificata nei canali consentiti."

In [ ]:
search_tool = SerperDevTool(n_results=2)
files = {
    'agents': '../config/agents.yaml',
    'tasks': '../config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

In [24]:
first_aid_chatbot = Agent(config=agents_config['first_aid_chatbot'], llm=llm)
first_aid_chatbot.tools=[youtube_channel_search_tool]

first_aid_response_task = Task(config=tasks_config['first_aid_response_task'], agent=first_aid_chatbot)
first_aid_response_task.tools=[youtube_channel_search_tool]

In [25]:
crew = Crew(
    agents=[first_aid_chatbot],
    tasks=[first_aid_response_task],
    process=Process.sequential,
)

# Kickoff del Crew
result = crew.kickoff(inputs={"query": "Mi ha punto un'ape, cosa fare?"})

2024-11-30 16:29:13,049 - 13148 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: First Aid Expert
## Task: Respond to user query with actionable first aid advice. Ensure safety, clarity, and relevance to the user's emergency. Your final answer must include a step-by-step guide and a reliable YouTube video link if a maneuver is recommended. This is the: Mi ha punto un'ape, cosa fare?



# Agent: First Aid Expert
## Thought: Thought: In caso di puntura di ape, è importante rimuovere il pungiglione al più presto e ridurre il gonfiore e il dolore. È anche importante monitorare le reazioni allergiche.
## Using tool: youtube_channel_search_tool
## Tool Input: 
"{\"query\": \"bee sting first aid\", \"max_results\": 1}"
## Tool Output: 
https://www.youtube.com/watch?v=kYdFTf4tY1I


# Agent: First Aid Expert
## Final Answer: 
Se sei stato punto da un'ape, segui questi passaggi:

1. Rimuovi il pungiglione il più presto possibile. Non schiacciare il pungiglione con le dita o con una pinzetta, poiché ciò può rilasciare più veleno nella pelle.
2. Lava la zona con acqua

In [26]:
print(result)

Se sei stato punto da un'ape, segui questi passaggi:

1. Rimuovi il pungiglione il più presto possibile. Non schiacciare il pungiglione con le dita o con una pinzetta, poiché ciò può rilasciare più veleno nella pelle.
2. Lava la zona con acqua e sapone.
3. Applica un impasto di bicarbonato di sodio e acqua per ridurre il gonfiore.
4. Applica un impacco freddo o un ghiaccio avvolto in un panno per ridurre il dolore e il gonfiore.
5. Prendi un antidolorifico come il paracetamolo o l'ibuprofene per ridurre il dolore.
6. Monitora le reazioni allergiche, come gonfiore, prurito o difficoltà respiratorie. Se noti alcuna di queste reazioni, chiama subito un servizio di emergenza.

Ulteriori informazioni su come gestire le punture di ape possono essere trovate nel seguente video: https://www.youtube.com/watch?v=kYdFTf4tY1I (First Aid for Life).


In [4]:
import pandas as pd
data_path = "../data/questions_samples/Domande_Primo_Soccorso.xlsx"
domande_test = pd.read_excel(data_path)
domande_test

,Domanda
0,Ho un amico che è appena svenuto e non si risv...
1,"Una persona è svenuta davanti a me, ma respira..."
2,Una persona sta respirando male e sembra che s...
3,"Mio padre non riesce più a respirare, ma è cos..."
4,Un mio amico si è tagliato con un coltello e s...
5,Ho appena visto una persona cadere e ha sbattu...
6,Mio fratello è caduto a terra e sembra avere d...
7,Un uomo si è accasciato a terra e non risponde...
8,Un bambino sta soffocando mentre mangiava. Cos...
9,Il mio neonato ha smesso di respirare all'impr...
